<a href="https://colab.research.google.com/gist/real-jiakai/33b3719ffb9bfe0e9a5a887dc1684800/bert_imdb_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 检查当前GPU信息
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-563f4d3d-25ad-7be2-e1d8-d9774be2688c)


In [2]:
# 安装python库
%pip install transformers datasets
%pip install transformers[torch] -U
%pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 33.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [3]:
from transformers import BertTokenizer, AutoModelForMaskedLM, Trainer, TrainingArguments
from datasets import load_dataset
import torch

In [4]:
# 清空GPU缓存
torch.cuda.empty_cache()

In [5]:
# 初始化tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [6]:
# 加载并处理IMDb数据集
def tokenize_and_prepare_labels(examples):
    # 对文本进行tokenization
    tokenized_inputs = tokenizer(examples['text'], padding='max_length', truncation=True)
    # BERT预训练期间需要labels字段，其内容是input_ids的副本
    tokenized_inputs['labels'] = tokenized_inputs['input_ids'].copy()
    return tokenized_inputs

dataset = load_dataset('imdb', split='train')
tokenized_datasets = dataset.map(
    tokenize_and_prepare_labels,
    batched=True,
    load_from_cache_file=False
)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# 初始化模型
model = AutoModelForMaskedLM.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
# 设置训练参数
training_args = TrainingArguments(
    output_dir='/content/drive/My Drive/Colab Notebooks/imdb_sentiment_analysis_with_bert/results',  # 确保路径正确
    num_train_epochs=3,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/content/drive/My Drive/Colab Notebooks/imdb_sentiment_analysis_with_bert/logs',
    logging_steps=10,
    evaluation_strategy="no",
    save_strategy="epoch",
    fp16=True,
    load_best_model_at_end=False,  # 因为没有评估，所以这里设置为False
)

In [10]:
# 初始化Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
)

In [11]:
# 开始训练
trainer.train()

Step,Training Loss
10,10.127500
20,8.565700
30,7.492900
40,6.507000
50,5.077100
60,4.163800
70,2.865000
80,2.213300
90,1.645900
100,1.221100


TrainOutput(global_step=9375, training_loss=0.055840135695291684, metrics={'train_runtime': 1262.6419, 'train_samples_per_second': 59.399, 'train_steps_per_second': 7.425, 'total_flos': 1.974036096e+16, 'train_loss': 0.055840135695291684, 'epoch': 3.0})

In [12]:
# 保存训练好的模型
trainer.save_model("/content/drive/My Drive/Colab Notebooks/imdb_sentiment_analysis_with_bert/trained_model")  # 保存到Google Drive